folowing the advice on i hate notebooks only move forward.

1. current inventory - make a table
2.  pending - make a table
3. make a list of all omb control numbers in inventory
4. make a list of all omb control numbers in pending
5. combine those lists
6. search the federal register for all those omb control numbers 
7.  save those to a list
8. create a table of all the notices
9. search for all the new notices for dol going back a year
10. use the inventory to make sure that collections that were new and approved are all in that list of notices
11. use the pending to make sure the collections that are new and submitted are in that list.

finding open loops:

12. what is :
            1. in inventory
            2. expiring in the next year.
            3. does not have a 60 day notice published within the last 9 months.
  
13. what is :
            1. in inventory
            2. expiring in the next year.
            3. has a 60 day notice published within the last 9 months.
            4. has not been submitted to oira
            
14. What are new collections that have had 60 day notices published in the last year that have not been submitted to oira?

how can we do this on a daily basis? ( automated / semi automated)

how can we add data to find when the agency sends the package to ocio? to show the real cycle time?

notes 

try an find the last published notice date for all the omb control numbers in inventory

In [1]:
import pycurl
import requests
import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup
import json
import pycurl
import requests
import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import shutil


dol_agency_codes = [
1220,
1225,
1205,
1219,
1218,
1250,
1245,
1240,
1293,
1290,
1235,
1210]

In [2]:
def get_xpath_strings(dol_agency_codes):       
    xpath_strings = []
    xpath_string_prefix = '//InformationCollectionRequest[.//AgencyCode/text()[. = "'
    xpath_string_suffix = '"]]'
    
    for number in dol_agency_codes:
        new_string = xpath_string_prefix+str(number)+xpath_string_suffix
        xpath_strings.append(new_string) 
    
    return(xpath_strings)
    

def inventory_to_list(file,dol_agency_codes):
    xpath_strings= get_xpath_strings(dol_agency_codes)
    
    tree = etree.parse(file)
    root = tree.getroot()
    all_results = []

    for string in xpath_strings:
      results = []
      req = root.xpath(string)
      for request in req:       
        try:
         results.append({
           "omb_control_number": request.xpath('./OMBControlNumber//text()')[0],
           "icr_reference_number": request.xpath('./ICRReferenceNumber//text()')[0],
           "Title": request.xpath('./Title//text()')[0],
           #"Abstract": request.xpath('./Abstract//text()')[0],
           "AgencyCode": str(request.xpath('./AgencyCode//text()')[0]).strip(),
           "ICRTypeCode": str(request.xpath('./ICRTypeCode//text()')[0]).strip(),
           "expiration_date": str(request.xpath('./Expiration/ExpirationDate//text()')[0]).strip(),
           "ICR_status": str(request.xpath('./ICRStatus//text()')[0]).strip()})
        except:
            results.append({
              "omb_control_number": request.xpath('./OMBControlNumber//text()')[0],
              "icr_reference_number": request.xpath('./ICRReferenceNumber//text()')[0],
              "Title": request.xpath('./Title//text()')[0],
              #"Abstract": request.xpath('./Abstract//text()')[0],
              "AgencyCode": str(request.xpath('./AgencyCode//text()')[0]).strip(),
              "ICRTypeCode": str(request.xpath('./ICRTypeCode//text()')[0]).strip(),
              "expiration_date": str(request.xpath('./Expiration/ExpirationDate//text()')[0]).strip(),
              "ICR_status": "None"})
            
      all_results.append(results) 
    return(all_results)
    
    
    
def inventory_list_to_table(file,all_results):    
    dol_inventory_row_list = []
    
    for list in all_results:
      for row in list:
        dol_inventory_row_list.append(row)
    
    dol_inventory_table = pd.DataFrame(dol_inventory_row_list)
    dol_inventory_table['file_date'] = file
    return(dol_inventory_table)    




def create_pending_df_from_xml(file):
    tree = etree.parse(file)
    root = tree.getroot()
    req = root.xpath('//InformationCollectionRequest')
    results = []
    for request in req:
        try:
            results.append({
                "omb_control_number": request.xpath('./OMBControlNumber//text()')[0],
                "icr_reference_number": request.xpath('./ICRReferenceNumber//text()')[0],
                "AgencyCode": str(request.xpath('./AgencyCode//text()')[0]).strip(),
                "Title": request.xpath('./Title//text()')[0],
                "submission_date": str(request.xpath('./SubmissionDate/Date//text()')[0]).strip()})
        except:
            pass
   
    df = pd.DataFrame(results)
    df['Agency'] = df['AgencyCode'].str[:2]
    return(df)


def dol_pending(pending_df):
    dol_pending = pending_df[pending_df['Agency']=='12']
    return(dol_pending)





current inventory - make a table

In [3]:
# first get the file 
os.chdir('/home/jupyter-ed/projects/pra/data/inventory')

In [4]:
os.listdir()

['2020-07-01_inventory.xml',
 'new_in_pending_today_table.csv',
 '2020-07-06_inventory.xml',
 '2020-07-10_inventory.xml',
 'pending.csv',
 '.ipynb_checkpoints',
 '2020-06-23_inventory.xml',
 '2020-07-02_inventory.xml',
 '2020-06-25_inventory.xml',
 '2020-06-28_inventory.xml',
 '2020-07-18_inventory.xml',
 '2020-07-29_inventory.xml',
 '2020-07-16_inventory.xml',
 '2020-07-26_inventory.xml',
 '2020-07-14_inventory.xml',
 '2020-07-15_inventory.xml',
 '2020-07-11_inventory.xml',
 '2020-07-28_inventory.xml',
 '2020-07-24_inventory.xml',
 '2020-07-17_inventory.xml',
 '2020-07-12_inventory.xml',
 '2020-07-08_inventory.xml',
 '2020-07-25_inventory.xml',
 '2020-07-13_inventory.xml',
 '2020-06-29_inventory.xml',
 '2020-07-20_inventory.xml',
 '2020-07-09_inventory.xml',
 '2020-07-30_inventory.xml',
 '2020-06-27_inventory.xml',
 '2020-07-21_inventory.xml',
 '2020-07-23_inventory.xml',
 '2020-06-24_inventory.xml',
 '2020-07-05_inventory.xml',
 '2020-07-19_inventory.xml',
 '2020-07-22_inventory.xml'

In [5]:
inventory_file = '2020-07-27_inventory.xml'

In [6]:
xpath_strings = get_xpath_strings(dol_agency_codes)

In [7]:
inventory_list =inventory_to_list(inventory_file,dol_agency_codes)

In [8]:
inventory_df = inventory_list_to_table(inventory_file,inventory_list)

pending - make a table

In [9]:
# first get the file 
os.chdir('/home/jupyter-ed/projects/pra/data/pending')

In [10]:
os.listdir()

['2020-07-08_pending.xml',
 '2020-07-14_pending.xml',
 '2020-07-12_pending.xml',
 '2020-07-30_pending.xml',
 '2020-07-13_pending.xml',
 '.ipynb_checkpoints',
 '2020-07-29_pending.xml',
 '2020-07-04_pending.xml',
 '2020-07-09_pending.xml',
 '2020-07-23_pending.xml',
 '2020-07-05_pending.xml',
 '2020-07-11_pending.xml',
 '2020-07-16_pending.xml',
 '2020-07-15_pending.xml',
 '2020-06-25_pending.xml',
 '2020-06-24_pending.xml',
 '2020-06-28_pending.xml',
 '2020-07-02_pending.xml',
 '2020-07-24_pending.xml',
 '2020-07-22_pending.xml',
 '2020-07-17_pending.xml',
 '2020-06-29_pending.xml',
 '2020-07-06_pending.xml',
 '2020-07-21_pending.xml',
 '2020-06-27_pending.xml',
 '2020-07-26_pending.xml',
 '2020-07-28_pending.xml',
 '2020-07-27_pending.xml',
 '2020-07-01_pending.xml',
 '2020-07-18_pending.xml',
 '2020-07-19_pending.xml',
 '2020-07-20_pending.xml',
 '2020-07-10_pending.xml',
 '2020-07-25_pending.xml',
 '2020-06-23_pending.xml']

In [11]:
pending_file ='2020-07-27_pending.xml'

In [12]:
pending_df_all = create_pending_df_from_xml(pending_file)


In [13]:
pending_df_dol = dol_pending(pending_df_all)


In [14]:
pending_df_dol

,omb_control_number,icr_reference_number,AgencyCode,Title,submission_date,Agency
434,1220-0163,202006-1220-004,1220,Consumer Price Index Housing Survey,2020-06-29-04:00,12
435,1220-0109,202004-1220-003,1220,National Longitudinal Survey of Youth 1979,2020-06-17-04:00,12
436,1220-0039,202002-1220-001,1220,Consumer Price Index Commodities and Services ...,2020-07-21-04:00,12
437,1210-0132,202003-1210-007,1210,Default Investment Alternatives under Particip...,2020-05-27-04:00,12
438,1210-0090,202003-1210-006,1210,Disclosures for Participant Directed Individua...,2020-05-27-04:00,12
439,1210-0039,202003-1210-004,1210,Summary Plan Description Requirements Under th...,2020-05-27-04:00,12
440,1210-0126,202003-1210-003,1210,Defined Benefit Plan Annual Funding Notice,2020-05-27-04:00,12
441,1210-0121,202003-1210-002,1210,Consent to Receive Employee Benefit Plan Discl...,2020-05-27-04:00,12
442,1205-0528,202004-1205-005,1205,American Apprenticeship Initiative Grants,2020-06-24-04:00,12
443,1205-0134,201912-1205-004,1205,Agricultural Recruitment System Forms Affectin...,2020-05-04-04:00,12


3. make a list of all omb control numbers in inventory


In [15]:
pending_omb_cntrl_number_list = pending_df_dol.omb_control_number.to_list()

In [16]:
len(pending_omb_cntrl_number_list)

47

4. make a list of all omb control numbers in pending


In [17]:
inventory_omb_cntrl_number_list = inventory_df.omb_control_number.to_list()



In [18]:
len(inventory_omb_cntrl_number_list)

474

In [19]:
icr_list = inventory_omb_cntrl_number_list

5. combine those lists


In [20]:
icr_list.extend(pending_omb_cntrl_number_list)

In [21]:
len(icr_list)

521

In [22]:
icr_list[-3]

'1235-0015'

example
https://www.federalregister.gov/api/v1/documents.json?conditions%5Bagencies%5D%5B%5D=labor-department&conditions%5Bterm%5D=  
1235-0015  
&conditions%5Btype%5D%5B%5D=NOTICE&order=newest"  


6. search the federal register for all those omb control numbers 
7.  save those to a list
8. create a table of all the notices


In [23]:
folder_name = datetime.datetime.now().strftime('notice_%Y-%m-%d')   
os.chdir('/home/jupyter-ed/projects/pra/scripts/EDA')


# define the name of the directory to be created
os.mkdir(folder_name)
os.chdir(folder_name)

In [24]:
def omb_cntrl_number_frn_link(omb_control_number):
    prefix = "https://www.federalregister.gov/api/v1/documents.json?conditions%5Bagencies%5D%5B%5D=labor-department&conditions%5Bterm%5D="
    suffix= "&conditions%5Btype%5D%5B%5D=NOTICE"
    query_url = prefix + omb_control_number + suffix
    return(query_url)


In [25]:
def get_omb_frn(query_url,omb_control_number):
    fr_request = requests.get(query_url)
    fr_json = fr_request.json()
    json_file_name = 'notices_'+omb_control_number+'.json'
    with open(json_file_name, 'w') as f:
        json.dump(fr_json, f)

In [26]:
test_omb_number ='1235-0015'
    

In [27]:
test_link = omb_cntrl_number_frn_link(test_omb_number)

In [28]:
test_link

'https://www.federalregister.gov/api/v1/documents.json?conditions%5Bagencies%5D%5B%5D=labor-department&conditions%5Bterm%5D=1235-0015&conditions%5Btype%5D%5B%5D=NOTICE'

In [29]:
get_omb_frn(test_link,test_omb_number)

for omb_number in icr_list:
    omb_link = omb_cntrl_number_frn_link(omb_number)
    get_omb_frn(omb_link,omb_number)
    

read in the json files and create a table 

In [30]:
test_link

'https://www.federalregister.gov/api/v1/documents.json?conditions%5Bagencies%5D%5B%5D=labor-department&conditions%5Bterm%5D=1235-0015&conditions%5Btype%5D%5B%5D=NOTICE'

In [31]:
fr_request = requests.get(test_link)
fr_json = fr_request.json()

In [32]:
fr_json

{'count': 10,
 'description': "Documents matching '1235-0015', from Labor Department, and of type Notice",
 'total_pages': 1,
 'results': [{'title': "Agency Information Collection Activities; Submission for OMB Review; Comment Request; Report of Construction Contractor's Wage Rates",
   'type': 'Notice',
   'abstract': 'The Department of Labor (DOL) is submitting this Wage and Hour Division (WHD)-sponsored information collection request (ICR) to the Office of Management and Budget (OMB) for review and approval in accordance with the Paperwork Reduction Act of 1995 (PRA). Public comments on the ICR are invited.',
   'document_number': '2020-11458',
   'html_url': 'https://www.federalregister.gov/documents/2020/05/28/2020-11458/agency-information-collection-activities-submission-for-omb-review-comment-request-report-of',
   'pdf_url': 'https://www.govinfo.gov/content/pkg/FR-2020-05-28/pdf/2020-11458.pdf',
   'public_inspection_pdf_url': 'https://s3.amazonaws.com/public-inspection.federal

In [33]:
fr_json['results'][0].keys()

dict_keys(['title', 'type', 'abstract', 'document_number', 'html_url', 'pdf_url', 'public_inspection_pdf_url', 'publication_date', 'agencies', 'excerpts'])

import json

with open('data.txt') as json_file:
    data = json.load(json_file)

In [34]:
os.listdir()

['notices_1235-0015.json',
 '.ipynb_checkpoints',
 'pending_eda.ipynb',
 'notice_2020-07-29',
 'html',
 'Untitled.ipynb',
 'history_eda.ipynb',
 'connect_to_postgres_database_and_add_a_df_as_table.ipynb',
 'todo_for_open_loop_report.ipynb',
 'notice_2020-07-28']

In [35]:
file='notices_1235-0029.json'

In [36]:
file[8:17]

'1235-0029'

In [37]:
with open(file) as json_file:
    data = json.load(json_file)
    

FileNotFoundError: [Errno 2] No such file or directory: 'notices_1235-0029.json'

In [ ]:
data['results'][0].keys()

In [ ]:
data['results'][0]['title']

rows = []

for file in os.listdir():
    with open(file) as json_file:
        data = json.load(json_file)
        if data['count'] > 0:
            notice=data['results'][0]
            row = [file[8:17],notice['title'],notice['document_number'],notice['html_url'],notice['publication_date']]
            rows.append(row)
        else:
            row = [file[8:17],'problem','problem','problem','problem']

In [ ]:
file_list =os.listdir()

In [ ]:
file_list

In [ ]:
file_list.remove('.ipynb_checkpoints')
file_list.remove('pending_eda.ipynb')

In [ ]:
rows = []

for file in file_list:
    print(file)
    with open(file) as json_file:
        data = json.load(json_file)
        if data['count'] > 0:
            notice=data['results'][0]
            row = [file[8:17],notice['title'],notice['document_number'],notice['html_url'],notice['publication_date']]
            rows.append(row)
        else:
            row = [file[8:17],'problem','problem','problem','problem']

In [ ]:
rows

In [ ]:
notice_dataframe = pd.DataFrame(rows)

In [ ]:
notice_dataframe.columns = ['omb_control_number','title','document_number','html_url','publication_date']

In [ ]:
notice_dataframe

finding the open loops that are new collections is its own project, going to finish finding the "open loops" for current inventory 
first 

these to do tasks are going to be after this project 
9. search for all the new notices for dol going back a year
10. use the inventory to make sure that collections that were new and approved are all in that list of notices
11. use the pending to make sure the collections that are new and submitted are in that list.

In [ ]:
inventory_df['expiration_date'] = pd.to_datetime(inventory_df['expiration_date'])


In [ ]:
 inventory_df[(inventory_df['expiration_date'] < '2021-07-27') ]['omb_control_number'].to_list()


In [ ]:
expire_in_next_year_df = inventory_df[(inventory_df['expiration_date'] < '2021-07-27') ]


In [ ]:
notice_dataframe['publication_date'] = pd.to_datetime(notice_dataframe['publication_date'])



In [ ]:
expire_in_next_year_not_in_pending_df = expire_in_next_year_df[~expire_in_next_year_df.omb_control_number.isin(pending_omb_cntrl_number_list)]

not sure where to put the cut off, but im going to start with a year out, for all the omb_control_numbers that havent had a notice pu
published in the last year.

In [ ]:
published_in_the_last_year_df = notice_dataframe[(notice_dataframe['publication_date'] > '2019-07-27')]



In [ ]:
published_in_the_last_year_list = published_in_the_last_year_df.omb_control_number.to_list()

In [ ]:
expire_in_next_year_no_recent_notice=expire_in_next_year_not_in_pending_df[~expire_in_next_year_not_in_pending_df.omb_control_number.isin(published_in_the_last_year_list)]

In [ ]:
expire_in_next_year_no_recent_notice.sort_values('expiration_date').head(30)

In [ ]:
expire_in_next_year_no_recent_notice.count()

In [ ]:
expire_in_next_year_no_recent_notice['expire_month'] = expire_in_next_year_no_recent_notice['expiration_date'].dt.strftime('%Y-%m')

In [ ]:
expire_in_next_year_no_recent_notice.groupby('expire_month')['expire_month'].count()

In [ ]:
expire_in_next_year_not_in_pending_df['expire_month'] = expire_in_next_year_not_in_pending_df['expiration_date'].dt.strftime('%Y-%m')


expire_in_next_year_not_in_pending_df.groupby('expire_month')['expire_month'].count()

# Part 2

from the top, we have number 11 and 12 done.

14 is going to have to be its own project.

for now , the next step in this project is to create something that monitors the cycle time and lead time

to do this i need to use a database

so the next part will be connecting to the database and create the tables needed to monitor that information.

In [38]:
inventory_df

,omb_control_number,icr_reference_number,Title,AgencyCode,ICRTypeCode,expiration_date,ICR_status,file_date
0,1220-0008,201705-1220-001,Producer Price Index Survey,1220,Extension without change of a currently approv...,2020-10-31-04:00,Active,2020-07-27_inventory.xml
1,1220-0011,202006-1220-006,Report on Current Employment Statistics,1220,No material or nonsubstantive change to a curr...,2020-12-31-05:00,Active,2020-07-27_inventory.xml
2,1220-0012,201708-1220-001,Quarterly Census of Employment and Wages (QCEW),1220,Extension without change of a currently approv...,2021-02-28-05:00,Active,2020-07-27_inventory.xml
3,1220-0017,201712-1220-002,Local Area Unemployment Statistics Program,1220,Revision of a currently approved collection,2021-07-31-04:00,Active,2020-07-27_inventory.xml
4,1220-0025,201806-1220-002,International Price Program U.S. Import and Ex...,1220,Extension without change of a currently approv...,2022-05-31-04:00,Active,2020-07-27_inventory.xml
...,...,...,...,...,...,...,...,...
469,1210-0150,201803-1210-001,Coverage of Certain Preventive Services under ...,1210,Revision of a currently approved collection,2021-11-30-05:00,Active,2020-07-27_inventory.xml
470,1210-0153,201902-1210-001,Alternative Reporting Methods for Apprenticesh...,1210,Revision of a currently approved collection,2022-08-31-04:00,Active,2020-07-27_inventory.xml
471,1210-0158,201906-1210-004,Insurance and Annuity Contracts and Mutual Fun...,1210,Revision of a currently approved collection,2022-08-31-04:00,Active,2020-07-27_inventory.xml
472,1210-0160,201905-1210-001,Notice for Health Reimbursement Arrangements i...,1210,New collection (Request for a new OMB Control ...,2022-06-30-04:00,Active,2020-07-27_inventory.xml


In [39]:
os.chdir('/home/jupyter-ed/projects/pra/data/01_database')

In [40]:
import sqlite3

In [41]:
DBPATH = 'pradb.db'
conn = sqlite3.connect(DBPATH)
cursor = conn.cursor()

inventory_df.to_sql('inventory',con=conn,if_exists='replace')

In [42]:
inventory_df.columns

Index(['omb_control_number', 'icr_reference_number', 'Title', 'AgencyCode',
       'ICRTypeCode', 'expiration_date', 'ICR_status', 'file_date'],
      dtype='object')

DBPATH = 'pradb.db'
conn = sqlite3.connect(DBPATH)


pending_df_dol.to_sql('pending',con=conn,if_exists='replace')

In [43]:
conn.close()

In [44]:
# at oira


In [45]:
# number in inventory

In [46]:
'''
website_url = requests.get(url).text
      soup = BeautifulSoup(website_url,'lxml')
      ombcntrl_table = soup.find('table',{'class':'datatable'})
      rows = ombcntrl_table.findAll('tr')
      for row in rows:
        list_of_cells = []
        cells = row.findAll('td')
        if len(cells)>0:
          counter=0
          for cell in cells:
            cell_info = cell.getText(strip='True')
            list_of_cells.append(cell_info)
          info.append(list_of_cells)
        elif len(cells)<1:
          counter+=1
          
          
'''     

"\nwebsite_url = requests.get(url).text\n      soup = BeautifulSoup(website_url,'lxml')\n      ombcntrl_table = soup.find('table',{'class':'datatable'})\n      rows = ombcntrl_table.findAll('tr')\n      for row in rows:\n        list_of_cells = []\n        cells = row.findAll('td')\n        if len(cells)>0:\n          counter=0\n          for cell in cells:\n            cell_info = cell.getText(strip='True')\n            list_of_cells.append(cell_info)\n          info.append(list_of_cells)\n        elif len(cells)<1:\n          counter+=1\n          \n          \n"

In [47]:
expire_in_next_year_no_recent_notice

NameError: name 'expire_in_next_year_no_recent_notice' is not defined

In [48]:
notice_dataframe

NameError: name 'notice_dataframe' is not defined

In [49]:
DBPATH = 'pradb.db'
conn = sqlite3.connect(DBPATH)
cursor = conn.cursor()

In [50]:


notice_dataframe.to_sql('notices',con=conn,if_exists='replace')

NameError: name 'notice_dataframe' is not defined

In [51]:
notice_dataframe

NameError: name 'notice_dataframe' is not defined

In [52]:
conn.close()

In [53]:
inventory_df.columns

Index(['omb_control_number', 'icr_reference_number', 'Title', 'AgencyCode',
       'ICRTypeCode', 'expiration_date', 'ICR_status', 'file_date'],
      dtype='object')

In [54]:
pending_df_dol.columns

Index(['omb_control_number', 'icr_reference_number', 'AgencyCode', 'Title',
       'submission_date', 'Agency'],
      dtype='object')

In [55]:
inventory_df.icr_reference_number.values

array(['201705-1220-001', '202006-1220-006', '201708-1220-001',
       '201712-1220-002', '201806-1220-002', '201810-1220-002',
       '201703-1220-001', '202001-1220-002', '201710-1220-001',
       '202004-1220-007', '201903-1220-001', '202004-1220-005',
       '201706-1220-002', '201902-1220-001', '201706-1220-003',
       '202004-1220-001', '201906-1220-002', '202003-1220-002',
       '201708-1220-004', '202004-1220-004', '201903-1220-002',
       '201806-1220-003', '201712-1220-003', '201711-1220-003',
       '201907-1220-001', '201904-1220-003', '202001-1220-003',
       '201805-1220-004', '201812-1220-002', '202005-1220-002',
       '201902-1220-002', '202004-1220-008', '202005-1220-001',
       '202006-1220-001', '202006-1220-005', '201803-1225-001',
       '201601-1225-004', '201709-1225-001', '201905-1225-001',
       '201706-1225-001', '201701-1225-001', '201705-1225-015CF',
       '201912-1225-001CF', '201902-1225-003CF', '201902-1225-002CF',
       '201912-1225-002CF', '201

In [59]:
pd.DataFrame(inventory_df.icr_reference_number.values,columns=['icr_reference_number']).to_csv('icr_numbers.csv')

In [60]:
os.getcwd()

'/home/jupyter-ed/projects/pra/data/01_database'

In [62]:
len(list(inventory_df.icr_reference_number.values))

474